In [1]:
import yt
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import pickle as pickle
from unyt import g,cm
from numpy.linalg import eig

box_A_root = '/data/gnedin/REI/D/Cai.B40.N256L2.sf=1_uv=0.15_bw=10_res=100.WC1/C/'
hc = pd.read_csv(box_A_root+'a=0.1453/hpropsRS.res', names=['Mvir','Mstar','Z','SFR','luminosity','px','py','pz','Rvir'],delim_whitespace=True,index_col=False)
ds = yt.load('/data/gnedin/REI/D/Cai.B40.N256L2.sf=1_uv=0.15_bw=10_res=100.WC1/C/rei40c1_a0.1453/rei40c1_a0.1453.art')
p = ds.artio_parameters
abox = p["abox"][0]
auni = p["auni"][0]
scale = auni/abox
##difference is llyman - the galaxies distances
data = []

def open_data():
    with open('peak_locations_box_c.pkl', 'rb') as f:
        peaks = pickle.load(f)
    return peaks


def calculation(peaks):
    # sp = ds.sphere(ds.arr((peaks[2][0]/scale,peaks[2][1]/scale,peaks[2][2]/scale), 'Mpccm/h'), (.026/scale, "Mpccm/h"))
    I = []
    for i in peaks:
        sp = ds.sphere(ds.arr((i[0]/scale,i[1]/scale,i[2]/scale), 'Mpccm/h'), (0.026/scale, "Mpccm/h"))
    # sp = ds.sphere(ds.arr((peaks[0][0]/scale,peaks[0][1]/scale,peaks[0][2]/scale), 'Mpccm/h'), (0.026/scale, "Mpccm/h"))  
        x = sp[("gas","x")] - (sp.center.to('cm'))[0]
        y = sp[("gas","y")] - (sp.center.to('cm'))[1]
        z = sp[("gas","z")] - (sp.center.to('cm'))[2]
        volume = sp[('gas','dx')] *sp[('gas','dy')]*sp[('gas','dz')]
        ix_temp = np.multiply(sp[('gas','density')]*volume,(np.power(y,2) + np.power(z,2)))
        iy_temp = np.multiply(sp[('gas','density')]*volume,(np.power(x,2) + np.power(z,2)))
        iz_temp = np.multiply(sp[('gas','density')]*volume,(np.power(x,2) + np.power(y,2)))
        ixy_temp = np.multiply(sp[('gas','density')]*volume,np.multiply(x,y))
        ixz_temp =  np.multiply(sp[('gas','density')]*volume,np.multiply(x,z))
        iyz_temp = np.multiply(sp[('gas','density')]*volume,np.multiply(z,y))
        Ix = np.sum(ix_temp)
        Iy = np.sum(iy_temp)
        Iz = np.sum(iz_temp)
        Ixy = np.sum(ixy_temp)
        Ixz = np.sum(ixz_temp)
        Iyz = np.sum(iyz_temp)
        I.append(np.array([[[Ix,Ixy,Ixz],[Ixy,Iy,Iyz],[Ixz,Iyz,Iz]]]))
    return(I)


def convertSphereToCart(theta, phi):
    "converts a unit vector in spherical to cartesian, needed for getGalaxies"
    return np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])

def eigen_stuff(I):
    eigen_vectors = []
    eigen_value = []
    eigen_value_total = []
    eigen_vector_total = []
    #max
    # eigen_vectors_max = 0
    # eigen_value_max = 0
    for i in I:
        print(i)
        eigenvalue, eigenvector= eig(i)
        #min
        eigen_vectors_max = eigenvector[0]
        eigen_value_max = eigenvalue[0]
        for i,z in zip(eigenvalue, eigenvector):
            #change this to largest or smallest
            if(i < eigen_value_max):
                eigen_value_max = i
                eigen_vectors_max = z
        eigen_value_total.append(eigenvalue)
        eigen_vector_total.append(eigenvector)
        eigen_vectors.append(eigen_vectors_max)
        eigen_value.append(eigen_value_max)
    return eigen_vectors,eigen_value, eigen_vector_total,eigen_value_total


def unit_vector(vector):
    return vector/np.linalg.norm(vector)

def angle_between(v1,v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.dot(v1_u, v2_u)


# def _I_x(field,data):
#     #need to minus to get the right thing! from the original position
#     x = data[("gas","x")] 
#     y = data[("gas","y")] 
#     z = data[("gas","z")] 
#     return  np.multiply(data[('gas','density')],(np.power(y,2) + np.power(z,2)))
# ds.add_field(('gas','I_x'),_I_x,sampling_type = "cell",units = "g/cm",force_override=True)


# def _I_xy(field,data):
#     x = data[("gas","x")]
#     y = data[("gas","y")] 
#     z = data[("gas","z")] 
#     return  np.multiply(data[('gas','density')],np.multiply(x,y))
# ds.add_field(('gas','I_xy'),_I_xy,sampling_type = "cell",units = "g/cm",force_override=True)



def main():
    angle = []
    peaks = open_data()
    I = calculation(peaks)
    with open('locations_directions_box_c.pkl', 'rb') as f:
        locations = pickle.load(f)
    vector_total = []
    value_total = []
    vector_no_filter_total  = []
    value_no_filter_total =[]
    for i,z in zip(I,locations):
        vector,value,vector_total_,value_total_ = eigen_stuff(i)
        cartesian = convertSphereToCart(z[0],z[1])
        # angle.append(angle_between(cartesian,vector))
        vector_total.append(vector)
        value_total.append(value)
        vector_no_filter_total.append(vector_total_)
        value_no_filter_total.append(value_total_)
    # with open('angle.pkl', 'wb') as f:
    #     pickle.dump(angle, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open('vector.pkl', 'wb') as f:
        pickle.dump(vector_total, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open('value.pkl', 'wb') as f:
        pickle.dump(value_total, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open('vector_total.pkl', 'wb') as f:
        pickle.dump(vector_no_filter_total, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open('value_total.pkl', 'wb') as f:
        pickle.dump(value_no_filter_total, f, protocol=pickle.HIGHEST_PROTOCOL)

yt : [WARNING  ] 2023-04-28 10:04:35,435 DeltaDC != 0, which implies auni != abox. Be sure you understand which expansion parameter is appropriate for your use! (Gnedin, Kravtsov, & Rudd 2011)
yt : [INFO     ] 2023-04-28 10:04:35,507 Parameters: current_time              = 958007400.6400309 yr
yt : [INFO     ] 2023-04-28 10:04:35,508 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2023-04-28 10:04:35,509 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-04-28 10:04:35,510 Parameters: domain_right_edge         = [256. 256. 256.]
yt : [INFO     ] 2023-04-28 10:04:35,511 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2023-04-28 10:04:35,511 Parameters: current_redshift          = 5.881049592936443
yt : [INFO     ] 2023-04-28 10:04:35,512 Parameters: omega_lambda              = 0.6964000165462494
yt : [INFO     ] 2023-04-28 10:04:35,513 Parameters: omega_matter              = 0.3035999834537506
yt : [INFO     ] 2023-04-28 10:04:3

In [2]:
peaks = open_data()

In [4]:
I = []
for i in peaks[:10]:
    sp = ds.sphere(ds.arr((i[0]/scale,i[1]/scale,i[2]/scale), 'Mpccm/h'), (0.026/scale, "Mpccm/h"))
# sp = ds.sphere(ds.arr((peaks[0][0]/scale,peaks[0][1]/scale,peaks[0][2]/scale), 'Mpccm/h'), (0.026/scale, "Mpccm/h"))  
    x = sp[("gas","x")] - (sp.center.to('cm'))[0]
    y = sp[("gas","y")] - (sp.center.to('cm'))[1]
    z = sp[("gas","z")] - (sp.center.to('cm'))[2]
    volume = sp[('gas','dx')] *sp[('gas','dy')]*sp[('gas','dz')]
    ix_temp = np.multiply(sp[('gas','density')]*volume,(np.power(y,2) + np.power(z,2)))
    iy_temp = np.multiply(sp[('gas','density')]*volume,(np.power(x,2) + np.power(z,2)))
    iz_temp = np.multiply(sp[('gas','density')]*volume,(np.power(x,2) + np.power(y,2)))
    ixy_temp = np.multiply(sp[('gas','density')]*volume,np.multiply(x,y))
    ixz_temp =  np.multiply(sp[('gas','density')]*volume,np.multiply(x,z))
    iyz_temp = np.multiply(sp[('gas','density')]*volume,np.multiply(z,y))
    #solar mass * Mpc**2
    #cm**2*g *solarmass/g/a/mpc*h^(-1)
    Ix = np.sum(ix_temp)/1.989e+33/0.1453/3.086e24 *0.68
    Iy = np.sum(iy_temp)/1.989e+33/0.1453/3.086e24 *0.68
    Iz = np.sum(iz_temp)/1.989e+33/0.1453/3.086e24 *0.68
    Ixy = np.sum(ixy_temp)/1.989e+33/0.1453/3.086e24 *0.68
    Ixz = np.sum(ixz_temp)/1.989e+33/0.1453/3.086e24 *0.68
    Iyz = np.sum(iyz_temp)/1.989e+33/0.1453/3.086e24 *0.68
    I.append(np.array([[[Ix,Ixy,Ixz],[Ixy,Iy,Iyz],[Ixz,Iyz,Iz]]]))



yt : [INFO     ] 2023-04-28 10:05:32,040 Created 4 chunks for ARTIO
yt : [INFO     ] 2023-04-28 10:05:32,340 Created 2 chunks for ARTIO


In [5]:
with open('locations_directions_box_c.pkl', 'rb') as f:
    locations = pickle.load(f)
vector_total = []
value_total = []
vector_no_filter_total  = []
value_no_filter_total =[]
for i,z in zip(I,locations):
    vector,value,vector_total_,value_total_ = eigen_stuff(i)
    cartesian = convertSphereToCart(z[0],z[1])
    # angle.append(angle_between(cartesian,vector))
    vector_total.append(vector)
    value_total.append(value)
    vector_no_filter_total.append(vector_total_)
    value_no_filter_total.append(value_total_)

[[3.30946085e+28 8.80438625e+27 1.42142942e+28]
 [8.80438625e+27 4.02839625e+28 9.69064536e+27]
 [1.42142942e+28 9.69064536e+27 2.59112767e+28]]
[[ 9.29889079e+34  1.91218922e+31 -4.46038929e+27]
 [ 1.91218922e+31  1.54500544e+28 -1.87631250e+31]
 [-4.46038929e+27 -1.87631250e+31  9.29889075e+34]]
[[ 7.38574954e+28 -2.87613087e+28 -2.97559386e+28]
 [-2.87613087e+28  7.49876074e+28  3.29496728e+28]
 [-2.97559386e+28  3.29496728e+28  7.22180803e+28]]
[[ 3.38766656e+27 -5.05604984e+26 -9.02841569e+26]
 [-5.05604984e+26  4.55238671e+27  3.48542925e+26]
 [-9.02841569e+26  3.48542925e+26  4.71269376e+27]]
[[ 5.60310609e+27 -7.54194236e+26 -1.50912287e+26]
 [-7.54194236e+26  3.61293430e+27 -7.25634953e+26]
 [-1.50912287e+26 -7.25634953e+26  5.15885940e+27]]
[[ 3.86370229e+27 -1.66906940e+26  4.83180557e+25]
 [-1.66906940e+26  4.00918405e+27  4.79032564e+25]
 [ 4.83180557e+25  4.79032564e+25  3.52595483e+27]]
[[ 4.62882597e+27 -3.86226817e+26  6.22432268e+25]
 [-3.86226817e+26  5.51101253e+27 

In [7]:
value_no_filter_total

[[array([5.53223501e+28, 2.93748289e+28, 1.45926686e+28])],
 [array([7.73191186e+27, 9.29889199e+34, 9.29889032e+34])],
 [array([1.34701150e+29, 4.60568804e+28, 4.03051525e+28])],
 [array([2.87477676e+27, 5.49138955e+27, 4.28658072e+27])],
 [array([3.09647593e+27, 5.87223121e+27, 5.40619264e+27])],
 [array([4.11883064e+27, 3.77237153e+27, 3.50763900e+27])],
 [array([4.48206102e+27, 5.52341883e+27, 6.39873304e+27])],
 [array([5.07875253e+27, 4.53161770e+27, 3.99159714e+27])],
 [array([8.07229434e+27, 1.12192539e+28, 9.69450109e+27])],
 [array([3.17773010e+27, 5.33700768e+27, 5.71647568e+27])]]